## Дапаможныя функцыі для працы

In [1]:
import math
import numpy

def get_generic(a, R0, m):
    R = []
    generic = []
    for i in range(5000000):
        if len(R) == 0:
            R.append(a * R0 % m)
        else:
            R.append(a * R[i-1] % m)
        generic.append(R[i] / m)
        
    return generic

def update_dict(states, s1, s2, s3):
    state = str(s1) + str(s2) + str(s3)
    
    if state in states:
        states[state] += 1
    else:
        states[state] = 1
        
def print_calculated_parametrs():
    print("P_отк*   ", lose_number / task_number)
    print("P_бл*    ", block_number / tact_number)
    print("A*       ", made_number / tact_number * 60)
    print("Q*       ", made_number / task_number)
    print("L_оч_1*  ", numpy.average(kol_tasks['queue_1']))
    print("L_оч_2*  ", numpy.average(kol_tasks['queue_2']))
    print("L_оч*    ", numpy.average(kol_tasks['queue']))
    print("L_c*     ", numpy.average(kol_tasks['system']))
    print("W_оч_1*  ", numpy.average(time_tasks['queue_1']))
    print("W_оч_2*  ", numpy.average(time_tasks['queue_2']))
    print("W_оч*    ", numpy.average(time_tasks['queue_2']))
    print("W_c*     ", numpy.average(time_tasks['system']))
    print("Ккан*    ", kanal / tact_number) 

## Уваходныя параметры

In [62]:
t = 20 # хвілін у экспаненцыяльным размеркаванні
lyambda = 2 # цягніка ў гадзіну ў найпрасцейшым размеркаванні
nyu = 60 / t
pi = 1 - math.e ** (-nyu)
ro = 1 - lyambda / (60 * math.e ** (lyambda / 60))

print("pi = ", pi)
print("ro = ", ro)

pi =  0.950212931632136
ro =  0.9677594633172665


## Генерацыя размеркаванняў верагоднасцяў

In [37]:
pi_0 = get_generic(102191, 131, 203563)
    
ro_0 = get_generic(102191, 133, 203563)

## Імітацыя працы сістэмы

In [40]:
from random import random

def work_machine(max_trains_on_hill):
    global tact_number, states, block_number, lose_number
    global made_number, task_number, kol_tasks, time_tasks, kanal, queue1_time
    
    s1 = 0        # чэрга на знешніх шляхах
    s2 = 0        # чэрга на горцы
    s3 = 0        # абслугоўваюцца

    tact_number = 1500000

    states = {}

    block_number, lose_number = 0, 0
    made_number, task_number = 0, 0
    
    kol_tasks = {key: list([]) for key in ["queue_1", "queue_2", "queue", "system"]}
    time_tasks = {key: list([]) for key in ["queue_1", "queue_2", "system"]}

    kanal = 0

    index_1 = 0      # for ro,
    index_2 = 0      # for pi

    task = []

    task_3, task_2, task_1 = 0, 0, 0
    task_i = 0
    
    queue1_time = []
    
    for i in range(tact_number):
        update_dict(states, s1, s2, s3)
        
        kol_tasks['system'].append(s1 + s2 + s3)
        kol_tasks['queue_1'].append(s1)
        kol_tasks['queue_2'].append(s2)
        kol_tasks['queue'].append(s1 + s2)
        
        if s3 == 1:
            kanal += 1
            task[task_3][2] += 1

            if pi_0[index_2] > pi:
                s3 = 0
                made_number += 1

                time_tasks['system'].append(task[task_3][2]) 

                task_3 += 1

            index_2 += 1          
    
        if s2 > 0:
            for j in range(task_2, task_2+s2):
                task[j][1] += 1
                task[j][2] += 1

            if s3 == 0:
                s3 = 1
                s2 -= 1

                time_tasks['queue_2'].append(task[task_2][1])

                task_2 += 1
        
        if s1 > 0:        
            for j in range(task_1, task_1+s1):
                task[j][0] += 1
                task[j][2] += 1
                
            queue1_time.append(s1)
            
            if s2 < max_trains_on_hill:
                s2 += 1
                s1 -= 1

                time_tasks['queue_1'].append(task[task_1][0])

                task_1 += 1
        else:
            queue1_time.append(0)
        
        if ro_0[index_1] >= ro: 
            task.append([0, 0, 0, i]) 

            if s3 == 0:
                s3 = 1
                task_1 = task_i + 1            
                task_2 = task_i + 1
                time_tasks['queue_1'].append(task[task_i][0])
                time_tasks['queue_2'].append(task[task_i][1])
            elif s2 < max_trains_on_hill:
                s2 += 1
                task_1 = task_i + 1
                time_tasks['queue_1'].append(task[task_i][0])
            else:
                s1 += 1

            task_i += 1

            task_number += 1

        index_1 += 1  

## Імітацыя сістэмы пры 3 цягніках на гары

In [63]:
work_machine(max_trains_on_hill=2)

states

{'000': 528604,
 '001': 357910,
 '011': 224728,
 '021': 140750,
 '121': 88923,
 '221': 56357,
 '321': 36901,
 '421': 22540,
 '521': 15277,
 '621': 9204,
 '721': 6540,
 '821': 4039,
 '921': 2833,
 '1021': 1726,
 '1121': 1090,
 '1221': 646,
 '1321': 780,
 '1421': 369,
 '1521': 127,
 '1621': 140,
 '1721': 203,
 '1821': 179,
 '1921': 91,
 '2021': 43}

### Атрыманыя верагоднасці

In [64]:
for key, value in states.items():
    print (key, '\t', value / tact_number)

000 	 0.35240266666666664
001 	 0.23860666666666666
011 	 0.14981866666666666
021 	 0.09383333333333334
121 	 0.059282
221 	 0.03757133333333333
321 	 0.024600666666666667
421 	 0.015026666666666667
521 	 0.010184666666666667
621 	 0.006136
721 	 0.00436
821 	 0.0026926666666666666
921 	 0.0018886666666666667
1021 	 0.0011506666666666666
1121 	 0.0007266666666666667
1221 	 0.00043066666666666665
1321 	 0.00052
1421 	 0.000246
1521 	 8.466666666666666e-05
1621 	 9.333333333333333e-05
1721 	 0.00013533333333333333
1821 	 0.00011933333333333334
1921 	 6.0666666666666666e-05
2021 	 2.8666666666666668e-05


### Параметры сістэмы

In [65]:
print_calculated_parametrs()

P_отк*    0.0
P_бл*     0.0
A*        1.9332799999999999
Q*        1.0
L_оч_1*   0.468942
L_оч_2*   0.6681626666666667
L_оч*     1.1371046666666667
L_c*      1.784702
W_оч_1*   14.553773897210958
W_оч_2*   20.73665480427046
W_оч*     20.73665480427046
W_c*      55.38883141603906
Ккан*     0.6475973333333334


### Падлік сумарнай колькасці штрафу

In [66]:
kol_1 = sum(queue1_time)
q1 = queue1_time.copy()
    
kol_1 // 60 * 1000

11723000

## Імітацыя сістэмы пры 4 цягніках на гары

In [67]:
work_machine(max_trains_on_hill=3)

states

{'000': 528604,
 '001': 357910,
 '011': 224728,
 '021': 140750,
 '031': 88923,
 '131': 56357,
 '231': 36901,
 '331': 22540,
 '431': 15277,
 '531': 9204,
 '631': 6540,
 '731': 4039,
 '831': 2833,
 '931': 1726,
 '1031': 1090,
 '1131': 646,
 '1231': 780,
 '1331': 369,
 '1431': 127,
 '1531': 140,
 '1631': 203,
 '1731': 179,
 '1831': 91,
 '1931': 43}

### Атрыманыя верагоднасці

In [68]:
for key, value in states.items():
    print (key, '\t', value / tact_number)

000 	 0.35240266666666664
001 	 0.23860666666666666
011 	 0.14981866666666666
021 	 0.09383333333333334
031 	 0.059282
131 	 0.03757133333333333
231 	 0.024600666666666667
331 	 0.015026666666666667
431 	 0.010184666666666667
531 	 0.006136
631 	 0.00436
731 	 0.0026926666666666666
831 	 0.0018886666666666667
931 	 0.0011506666666666666
1031 	 0.0007266666666666667
1131 	 0.00043066666666666665
1231 	 0.00052
1331 	 0.000246
1431 	 8.466666666666666e-05
1531 	 9.333333333333333e-05
1631 	 0.00013533333333333333
1731 	 0.00011933333333333334
1831 	 6.0666666666666666e-05
1931 	 2.8666666666666668e-05


### Параметры сістэмы

In [69]:
print_calculated_parametrs()

P_отк*    0.0
P_бл*     0.0
A*        1.9332799999999999
Q*        1.0
L_оч_1*   0.30360333333333334
L_оч_2*   0.8335013333333333
L_оч*     1.1371046666666667
L_c*      1.784702
W_оч_1*   9.422432342961185
W_оч_2*   25.867996358520234
W_оч*     25.867996358520234
W_c*      55.38883141603906
Ккан*     0.6475973333333334


### Падлік сумарнай колькасці штрафу

In [70]:
kol_2 = sum(queue1_time)
q2 = queue1_time.copy()
    
kol_2 // 60 * 1000

7590000

## Падлік часу, калі акупіцца прыбаўка адного месца на гару

In [73]:
sum1 = 0
sum2 = 0
moment = 0

for i in range(len(q1)):
    sum1 += q1[i]
    sum2 += q2[i]
    if sum1 > sum2 == 60000:
        print(sum1 - sum2)
#     print(q1[i], q2[i], sum1, sum2, sum1 - sum2, (sum1 - sum2) / 60)
    
    if sum1 - sum2  >= 60000:
        moment = i
        break
        
print(i // 60 // 24, 'дзён')

26723
252 дзён
